In [51]:
# Gerekli kütüphaneleri import et
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

warnings.filterwarnings('ignore')

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Proje kök dizinini Python path'e ekle
import sys
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from src.config import *
from src.data_processing import load_arff_data

print("✅ Kütüphaneler başarıyla yüklendi!")
print(f"📁 Proje Dizini: {BASE_DIR}")
print(f"📁 İşlenmiş Veri Dizini: {PROCESSED_DATA_DIR}")

✅ Kütüphaneler başarıyla yüklendi!
📁 Proje Dizini: c:\Users\PC\Desktop\classification_project
📁 İşlenmiş Veri Dizini: c:\Users\PC\Desktop\classification_project\data\processed


## 1. Veri Yükleme ve Kontrol

Data Understanding aşamasında analiz ettiğimiz veriyi yüklüyoruz ve temel kontrollerini yapıyoruz.

In [52]:
print("="*80)
print("VERİ YÜKLEME")
print("="*80)

try:
    # ARFF dosyasını yükle
    df = load_arff_data(RAW_DATA_FILE)
    
    print(f"\n" + "="*80)
    print("VERİ ÖNİZLEME")
    print("="*80)
    display(df.head())
    
    print(f"\n" + "="*80)
    print("VERİ SETİ BİLGİLERİ")
    print("="*80)
    print(f"""
📊 Temel İstatistikler:
   • Toplam Satır: {df.shape[0]:,}
   • Toplam Sütun: {df.shape[1]}
   • Özellik Sayısı: {len(FEATURE_NAMES)}
   • Hedef Değişken: {TARGET_NAME}
   
📋 Sınıf Dağılımı:
   • Phishing (-1): {(df[TARGET_NAME] == -1).sum():,} ({(df[TARGET_NAME] == -1).sum() / len(df) * 100:.2f}%)
   • Legitimate (1): {(df[TARGET_NAME] == 1).sum():,} ({(df[TARGET_NAME] == 1).sum() / len(df) * 100:.2f}%)
""")
    
except Exception as e:
    print(f"❌ Hata: {e}")
    raise

VERİ YÜKLEME
✅ ARFF dosyası başarıyla yüklendi: Training Dataset.arff
   Boyut: 11,055 satır x 31 sütun

VERİ ÖNİZLEME


,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1



VERİ SETİ BİLGİLERİ

📊 Temel İstatistikler:
   • Toplam Satır: 11,055
   • Toplam Sütun: 31
   • Özellik Sayısı: 30
   • Hedef Değişken: Result

📋 Sınıf Dağılımı:
   • Phishing (-1): 4,898 (44.31%)
   • Legitimate (1): 6,157 (55.69%)



## 2. Tekrarlı Satırların Temizlenmesi

Data Understanding aşamasında **5,206 tekrarlı satır** tespit edilmişti. Bu tekrarları temizleyeceğiz.

**Neden Önemli?**
- Model overfitting yapabilir
- Test verisi kontamine olabilir (data leakage)
- Gerçek performans ölçümü yanıltıcı olur

In [53]:
print("\n" + "="*80)
print("TEKRARLI SATIRLARI TEMİZLEME")
print("="*80)

# data_processing modülünden fonksiyonu import et
from src.data_processing import remove_duplicates

# Tekrarları kaldır
df = remove_duplicates(df, keep='first', verbose=True)

print(f"\n✅ Veri temizleme tamamlandı!")
print(f"   Çalışmaya {df.shape[0]:,} satır ile devam ediyoruz.")


TEKRARLI SATIRLARI TEMİZLEME

TEKRARLI SATIRLARI TEMİZLEME

📊 Önceki Durum:
   • Toplam Satır: 11,055
   • Tekrarlı Satır: 5,206
   • Benzersiz Satır: 5,849

📊 Sonraki Durum:
   • Toplam Satır: 5,849
   • Kaldırılan Satır: 5,206
   • Veri Kaybı: 47.09%

✅ Temizleme tamamlandı!

✅ Veri temizleme tamamlandı!
   Çalışmaya 5,849 satır ile devam ediyoruz.


## 3. Yüksek Korelasyonlu Özelliklerin Kontrolü

**Multicollinearity (Çoklu Doğrusal Bağlantı)** problemi:
- Birbirine çok benzer özellikler modeli karıştırır
- Model yorumlanabilirliğini azaltır
- Gereksiz hesaplama maliyeti

**Strateji:**
- Korelasyon > 0.85 olan özellik çiftlerini tespit et
- Her çiftten hedef değişkenle korelasyonu düşük olanı çıkar

In [54]:
print("="*80)
print("YÜKSEK KORELASYONLU ÖZELLİKLERİN TESPİTİ")
print("="*80)

# data_processing modülünden fonksiyonu import et
from src.data_processing import find_high_correlation_features

# Yüksek korelasyonlu özellikleri bul
features_to_drop = find_high_correlation_features(
    df, FEATURE_NAMES, TARGET_NAME, threshold=0.8, verbose=True
)

print(f"\n📊 Özet:")
print(f"   • Başlangıç Özellik Sayısı: {len(FEATURE_NAMES)}")
print(f"   • Çıkarılacak Özellik: {len(features_to_drop)}")
print(f"   • Kalan Özellik: {len(FEATURE_NAMES) - len(features_to_drop)}")

YÜKSEK KORELASYONLU ÖZELLİKLERİN TESPİTİ

YÜKSEK KORELASYONLU ÖZELLİKLERİN TESPİTİ

⚠️  3 yüksek korelasyonlu çift tespit edildi (|r| ≥ 0.8):
   • Shortining_Service ↔ double_slash_redirecting: 0.8158
   • Favicon ↔ port: 0.8127
   • Favicon ↔ popUpWidnow: 0.9331

📋 Çıkarılacak Özelliklerin Belirlenmesi:
   (Her çiftten hedef değişkenle korelasyonu düşük olanı çıkarılacak)

   Özellik 1                      r(target)    Özellik 2                      r(target)    Çıkarılacak                   
   -------------------------------------------------------------------------------------------------------------------
   Shortining_Service             0.0762       double_slash_redirecting       0.0367       double_slash_redirecting      
   Favicon                        0.0255       port                           0.0142       port                          
   Favicon                        0.0255       popUpWidnow                    0.0225       popUpWidnow                   

✅ Çıkarılacak Ö

In [55]:
print("="*80)
print("GEREKSİZ ÖZELLİKLERİN ÇIKARILMASI")
print("="*80)

# data_processing modülünden fonksiyonu import et
from src.data_processing import remove_multicollinear_features

# Multicollinear özellikleri çıkar
df, FEATURE_NAMES_CLEANED = remove_multicollinear_features(
    df, FEATURE_NAMES, TARGET_NAME, threshold=0.8, verbose=True
)

print(f"\n✅ Veri temizleme tamamlandı!")
print(f"   Çalışmaya {len(FEATURE_NAMES_CLEANED)} özellik ile devam ediyoruz.")

GEREKSİZ ÖZELLİKLERİN ÇIKARILMASI

YÜKSEK KORELASYONLU ÖZELLİKLERİN TESPİTİ

⚠️  3 yüksek korelasyonlu çift tespit edildi (|r| ≥ 0.8):
   • Shortining_Service ↔ double_slash_redirecting: 0.8158
   • Favicon ↔ port: 0.8127
   • Favicon ↔ popUpWidnow: 0.9331

📋 Çıkarılacak Özelliklerin Belirlenmesi:
   (Her çiftten hedef değişkenle korelasyonu düşük olanı çıkarılacak)

   Özellik 1                      r(target)    Özellik 2                      r(target)    Çıkarılacak                   
   -------------------------------------------------------------------------------------------------------------------
   Shortining_Service             0.0762       double_slash_redirecting       0.0367       double_slash_redirecting      
   Favicon                        0.0255       port                           0.0142       port                          
   Favicon                        0.0255       popUpWidnow                    0.0225       popUpWidnow                   

✅ Çıkarılacak Özellikl

In [56]:
print("="*80)
print("ÖZELLİK SEÇİMİ DOĞRULAMASI")
print("="*80)

# Temizlenmiş veri setinin temel bilgileri
print(f"\n📊 Temizlenmiş Veri Seti Bilgileri:")
print(f"   • Toplam Satır: {df.shape[0]:,}")
print(f"   • Toplam Sütun: {df.shape[1]}")
print(f"   • Özellik Sayısı: {len(FEATURE_NAMES_CLEANED)}")
print(f"   • Hedef Değişken: {TARGET_NAME}")

# Sınıf dağılımı kontrolü
print(f"\n📋 Sınıf Dağılımı:")
class_dist_clean = df[TARGET_NAME].value_counts().sort_index()
for cls, count in class_dist_clean.items():
    label = CLASS_LABELS[cls]
    percentage = (count / len(df)) * 100
    print(f"   • {label} ({cls:2}): {count:,} ({percentage:.2f}%)")

# Dengesizlik oranı
imbalance_ratio_clean = class_dist_clean.max() / class_dist_clean.min()
print(f"\n⚖️  Dengesizlik Oranı: {imbalance_ratio_clean:.2f}:1")

# Temizlenmiş veri setinin korelasyon matrisi
print(f"\n🔍 Temizlenmiş Veri Seti Korelasyon Kontrolü:")
X_clean = df[FEATURE_NAMES_CLEANED]
corr_matrix_clean = X_clean.corr()

# Yüksek korelasyonlu çiftleri kontrol et (0.8+ threshold)
threshold_check = 0.8
high_corr_pairs_clean = []

for i in range(len(corr_matrix_clean.columns)):
    for j in range(i+1, len(corr_matrix_clean.columns)):
        if abs(corr_matrix_clean.iloc[i, j]) >= threshold_check:
            high_corr_pairs_clean.append({
                'Özellik 1': corr_matrix_clean.columns[i],
                'Özellik 2': corr_matrix_clean.columns[j],
                'Korelasyon': corr_matrix_clean.iloc[i, j]
            })

if len(high_corr_pairs_clean) > 0:
    print(f"\n⚠️  {len(high_corr_pairs_clean)} yüksek korelasyonlu çift kaldı (|r| ≥ {threshold_check}):")
    for pair in high_corr_pairs_clean:
        print(f"   • {pair['Özellik 1']} - {pair['Özellik 2']}: {pair['Korelasyon']:.4f}")
else:
    print(f"\n✅ {threshold_check} eşiğinde yüksek korelasyonlu çift kalmadı!")

# Hedef değişkenle korelasyonları kontrol et
print(f"\n📈 Hedef Değişkenle Korelasyonlar:")
target_corr_clean = df[FEATURE_NAMES_CLEANED + [TARGET_NAME]].corr()[TARGET_NAME].drop(TARGET_NAME)

# En yüksek pozitif ve negatif korelasyonlar
top_positive = target_corr_clean.nlargest(5)
top_negative = target_corr_clean.nsmallest(5)

print(f"\n   En Yüksek Pozitif Korelasyonlar:")
for feat, corr in top_positive.items():
    print(f"   • {feat:<30} {corr:.4f}")

print(f"\n   En Yüksek Negatif Korelasyonlar:")
for feat, corr in top_negative.items():
    print(f"   • {feat:<30} {corr:.4f}")

# Veri kalitesi kontrolü
print(f"\n🔍 Veri Kalitesi Kontrolü:")
print(f"   • Eksik Değer: {df.isnull().sum().sum()}")
print(f"   • Sonsuz Değer: {np.isinf(df.select_dtypes(include=[np.number])).sum().sum()}")
print(f"   • Tekrarlı Satır: {df.duplicated().sum()}")

# Bellek kullanımı
memory_usage = df.memory_usage(deep=True).sum() / 1024**2
print(f"   • Bellek Kullanımı: {memory_usage:.2f} MB")

print(f"\n" + "="*80)
print("SONUÇ")
print("="*80)
print(f"✅ Özellik seçimi doğrulaması tamamlandı!")
print(f"   • Başlangıç: {len(FEATURE_NAMES)} özellik")
print(f"   • Sonuç: {len(FEATURE_NAMES_CLEANED)} özellik")
print(f"   • Azalma: {len(FEATURE_NAMES) - len(FEATURE_NAMES_CLEANED)} özellik ({(len(FEATURE_NAMES) - len(FEATURE_NAMES_CLEANED))/len(FEATURE_NAMES)*100:.1f}%)")
print(f"   • Veri kalitesi: {'✅ İyi' if df.isnull().sum().sum() == 0 and df.duplicated().sum() == 0 else '⚠️ Kontrol gerekli'}")

ÖZELLİK SEÇİMİ DOĞRULAMASI

📊 Temizlenmiş Veri Seti Bilgileri:
   • Toplam Satır: 5,849
   • Toplam Sütun: 28
   • Özellik Sayısı: 27
   • Hedef Değişken: Result

📋 Sınıf Dağılımı:
   • Phishing (-1): 3,019 (51.62%)
   • Legitimate ( 1): 2,830 (48.38%)

⚖️  Dengesizlik Oranı: 1.07:1

🔍 Temizlenmiş Veri Seti Korelasyon Kontrolü:

✅ 0.8 eşiğinde yüksek korelasyonlu çift kalmadı!

📈 Hedef Değişkenle Korelasyonlar:

   En Yüksek Pozitif Korelasyonlar:
   • SSLfinal_State                 0.6932
   • URL_of_Anchor                  0.6791
   • Prefix_Suffix                  0.3811
   • having_Sub_Domain              0.2962
   • web_traffic                    0.2817

   En Yüksek Negatif Korelasyonlar:
   • Domain_registeration_length    -0.2186
   • Abnormal_URL                   -0.0900
   • Shortining_Service             -0.0762
   • HTTPS_token                    -0.0514
   • Redirect                       -0.0375

🔍 Veri Kalitesi Kontrolü:
   • Eksik Değer: 0
   • Sonsuz Değer: 0
   • Tek

In [57]:
print("="*80)
print("VERİ BÖLME (TRAIN-TEST SPLIT)")
print("="*80)

# data_processing modülünden fonksiyonu import et
from src.data_processing import split_data

# Veriyi böl
X_train, X_test, y_train, y_test = split_data(
    df, FEATURE_NAMES_CLEANED, TARGET_NAME, 
    test_size=TEST_SIZE, random_state=RANDOM_STATE, 
    stratify=True, verbose=True
)

print(f"\n✅ Veri bölme işlemi tamamlandı!")

VERİ BÖLME (TRAIN-TEST SPLIT)

VERİ BÖLME (TRAIN-TEST SPLIT)

📊 Veri Seti Bilgileri:
   • Toplam Veri: 5,849 satır
   • Özellik Sayısı: 27
   • Hedef Değişken: Result

⚙️  Split Parametreleri:
   • Train: %80
   • Test: %20
   • Random State: 42
   • Stratified: Evet

✅ Split tamamlandı!

📊 Train Set:
   • Boyut: 4,679 satır x 27 sütun
   • Sınıf -1: 2,415 (51.61%)
   • Sınıf  1: 2,264 (48.39%)

📊 Test Set:
   • Boyut: 1,170 satır x 27 sütun
   • Sınıf -1: 604 (51.62%)
   • Sınıf  1: 566 (48.38%)

✅ Veri bölme işlemi tamamlandı!


In [58]:
print("="*80)
print("İŞLENMİŞ VERİYİ KAYDETME")
print("="*80)

# data_processing modülünden fonksiyonu import et
from src.data_processing import save_processed_data

# Veriyi kaydet
save_processed_data(
    X_train, X_test, y_train, y_test,
    FEATURE_NAMES_CLEANED, TARGET_NAME,
    TRAIN_DATA_FILE, TEST_DATA_FILE,
    PROCESSED_DATA_DIR / "feature_names.txt",
    verbose=True
)

print(f"\n✅ İşlenmiş veriler diske kaydedildi ve hazır!")

İŞLENMİŞ VERİYİ KAYDETME

İŞLENMİŞ VERİYİ KAYDETME

✅ Veri dosyaları kaydedildi:
   • Train: c:\Users\PC\Desktop\classification_project\data\processed\train.csv
   • Test:  c:\Users\PC\Desktop\classification_project\data\processed\test.csv

📊 Dosya Boyutları:
   • Train: 4,679 satır x 28 sütun
   • Test:  1,170 satır x 28 sütun
   • Özellikler: c:\Users\PC\Desktop\classification_project\data\processed\feature_names.txt
   • Özellik Sayısı: 27

✅ Tüm dosyalar başarıyla kaydedildi!

✅ İşlenmiş veriler diske kaydedildi ve hazır!


## Data Preparation Özeti

### 1) Dosyalar ve Yollar
- Proje Dizini: `{{ BASE_DIR }}`
- Ham Veri: `{{ RAW_DATA_FILE }}`
- İşlenmiş Train (CSV): `{{ TRAIN_DATA_FILE }}`
- İşlenmiş Test (CSV): `{{ TEST_DATA_FILE }}`
- Özellik Listesi: `{{ PROCESSED_DATA_DIR }}/feature_names.txt`

---

### 2) Boyutlar
- Orijinal Veri: 11,055 satır x 31 sütun
- Temizlenmiş Veri: {len(df)} satır x {len(FEATURE_NAMES_CLEANED) + 1} sütun
- Train: {len(X_train)} satır x {X_train.shape[1] + 1} sütun (hedef dahil)
- Test: {len(X_test)} satır x {X_test.shape[1] + 1} sütun (hedef dahil)

---

### 3) Parametreler
- TEST_SIZE: `{TEST_SIZE}`
- RANDOM_STATE: `{RANDOM_STATE}`

---

### 4) Özellikler
- Başlangıç Özellikleri: {len(FEATURE_NAMES)}
- Kalan Özellikler: {len(FEATURE_NAMES_CLEANED)}
- Çıkarılanlar: {', '.join(sorted(set(FEATURE_NAMES) - set(FEATURE_NAMES_CLEANED))) if set(FEATURE_NAMES) - set(FEATURE_NAMES_CLEANED) else '-'}

---

### 5) Hedefle En Güçlü Korelasyonlar
- En Yüksek Pozitif (Top 5):
  {corr_final.nlargest(5).to_string()}

- En Yüksek Negatif (Top 5):
  {corr_final.nsmallest(5).to_string()}

---

### 6) Veri Kalitesi
- Eksik Değer: {df.isnull().sum().sum()}
- Sonsuz Değer: {np.isinf(df.select_dtypes(include=[np.number])).sum().sum()}
- Tekrarlı Satır: {df.duplicated().sum()}

---

### Sonuç
- Data preparation tamamlandı ve veriler kaydedildi.